# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
### conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')


In [3]:
### Get the students table
### students = pd.read_sql("SELECT * FROM Students;",conn) 

In [4]:
college = pd.ExcelFile("../data/college.xlsx")
departments = college.parse(0)
teachers = college.parse(1)
students = college.parse(2)
courses = college.parse(3)
classrooms = college.parse(4)

# Questions
 
 
## Q1. Count the number of students on each department

In [10]:
### Join tables of classrooms, courses and departments:
class_cour = pd.merge(classrooms, courses, on='CourseID', how='left')
dep_class_cour = pd.merge(departments, class_cour, on='DepartmentID', how='left')

## Remove duplicates and group by department:
result = dep_class_cour.drop_duplicates(subset=['StudentID','DepartmentID']).groupby('DepartmentName', as_index=False).agg({'StudentID':'count'})
result.columns = ['Department_name', 'Number_of_students']
result



,Department_name,Number_of_students
0,Arts,137
1,English,84
2,Science,200
3,Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [11]:
### Join tables of classrooms, courses and departments:
class_cour = pd.merge(classrooms, courses, on='CourseID', how='left')
dep_class_cour = pd.merge(departments, class_cour, on='DepartmentID', how='left')

### Find all courses in English dep and count students:
result = dep_class_cour.loc[(dep_class_cour['DepartmentName']=='English')].groupby('CourseName', as_index=True).agg({'StudentID':'count'})

## Add the 'total' row for the entire department:
result.loc['English_Total'] = dep_class_cour.loc[(dep_class_cour['DepartmentName']=='English')].drop_duplicates(subset=['StudentID','DepartmentName']).count()
result.columns = ['Number_of_students']
result


,Number_of_students
CourseName,
Advanced English,24
English Begginers,32
Proffesional English,38
English_Total,84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [12]:
### Join tables of classrooms, courses and departments:
class_cour = pd.merge(classrooms, courses, on='CourseID', how='left')
dep_class_cour = pd.merge(departments, class_cour, on='DepartmentID', how='left')

### Group the number of students per class and identify big and small classes:
CountStudents = dep_class_cour.loc[(dep_class_cour['DepartmentName']=='Science')].groupby('CourseName', as_index=False).agg({'StudentID':'count'})
CountStudents.columns = ['Course_name', 'Number_of_students']
CountStudents['Class_size'] = 'Small classrooms'
CountStudents.loc[CountStudents['Number_of_students']>21, 'Class_size']='Big classrooms'

### Group the big classes and small classes:
result=CountStudents.groupby('Class_size', as_index=False).agg({'Number_of_students':'count'})
result

,Class_size,Number_of_students
0,Big classrooms,10
1,Small classrooms,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [13]:
result = students.groupby('Gender', as_index=False).agg({'StudentID':'count'})
result.columns = ['Gender', 'Number_of_students']
result

,Gender,Number_of_students
0,F,165
1,M,115


# Q5. For which courses the percentage of male/female students is over 70%?

In [14]:
### Join tables of classrooms, courses and students:
class_cour = pd.merge(classrooms, courses, on='CourseID', how='left')
stu_class_cour = pd.merge(students, class_cour, on='StudentID', how='left')

### Create a table to show number of women in each course:
Females = stu_class_cour[stu_class_cour['Gender'] == 'F']
F = Females.groupby(['CourseID','CourseName'], as_index=False).agg({'Gender':'count'})
F.columns=['CourseID', 'Course_name', 'Women']

### Create a table to show number of men in each course:
Males = stu_class_cour[stu_class_cour['Gender'] == 'M']
M = Males.groupby(['CourseID','CourseName'], as_index=False).agg({'Gender':'count'})
M.columns=['CourseID','Course_name', 'Men']

### Merge the two tables to the result, showing number of men, women and ratio:
result = pd.merge(M, F, on=['Course_name', 'CourseID'], how='left')
result['ratio']=result['Women']/(result['Women']+result['Men'])*100

### Present courses with ratio over 70%:
result.loc[(result['ratio']>70)]



,CourseID,Course_name,Men,Women,ratio
21,22.0,Sculpture,7,17,70.833333
26,29.0,Tenis,5,13,72.222222


## Q6. For each department, how many students passed with a grades over 80?

In [20]:
### Join tables of classrooms, courses and departments:
class_cour = pd.merge(classrooms, courses, on='CourseID', how='left')
dep_class_cour = pd.merge(departments, class_cour, on='DepartmentID', how='left')

# Create a table to show number of students in each department:
AllStudents = dep_class_cour.drop_duplicates(subset=['StudentID','DepartmentID']).groupby('DepartmentName', as_index=False).agg({'StudentID':'count'})
AllStudents.columns=['Department_name', 'Number_of_students']

# Create a table to show number of top students in each department (grade over 80) and group by department:
TopStudents = dep_class_cour[dep_class_cour["degree"] > 80]
TopStudentsGrouped = TopStudents.drop_duplicates(subset=['StudentID','DepartmentID']).groupby('DepartmentName', as_index=False).agg({'StudentID':'count'})
TopStudentsGrouped.columns=['Department_name', 'Number_of_top_students']

### Merge the two tables to the result, showing number of all students, top students and ratio:
result = pd.merge(TopStudentsGrouped, AllStudents, on='Department_name', how='left')
result['ratio']=result.Number_of_top_students/result.Number_of_students*100
result.columns = ['Department_name', 'Number_of_top_students', 'Number_of_students', 'Top_students_ratio']
result

,Department_name,Number_of_top_students,Number_of_students,Top_students_ratio
0,Arts,61,137,44.525547
1,English,40,84,47.619048
2,Science,122,200,61.000000
3,Sport,39,81,48.148148


## Q7. For each department, how many students passed with a grades under 60?

In [17]:
### Join tables of classrooms, courses and departments:
class_cour = pd.merge(classrooms, courses, on='CourseID', how='left')
dep_class_cour = pd.merge(departments, class_cour, on='DepartmentID', how='left')

# Create a table to show number of students in each department:
AllStudents = dep_class_cour.drop_duplicates(subset=['StudentID','DepartmentID']).groupby('DepartmentName', as_index=False).agg({'StudentID':'count'})
AllStudents.columns=['Department_name', 'Number_of_students']

# Create a table to show number of low students in each department (grade under 60) and group by department:
LowStudents = dep_class_cour[dep_class_cour["degree"] < 60]
LowStudentsGrouped = LowStudents.drop_duplicates(subset=['StudentID','DepartmentID']).groupby('DepartmentName', as_index=False).agg({'StudentID':'count'})
LowStudentsGrouped.columns=['Department_name', 'Number_of_low_students']

### Merge the two tables to the result, showing number of all students, low students and ratio:
result = pd.merge(LowStudentsGrouped, AllStudents, on='Department_name', how='left')
result['ratio']=result.Number_of_low_students/result.Number_of_students*100
result.columns = ['Department_name', 'Number_of_low_students', 'Number_of_students', 'Low_students_ratio']

result

,Department_name,Number_of_low_students,Number_of_students,Low_students_ratio
0,Arts,3,137,2.189781
1,English,2,84,2.380952
2,Science,7,200,3.500000
3,Sport,1,81,1.234568


## Q8. Rate the teachers by their average student's grades (in descending order).

In [17]:
### Join tables of classrooms, courses and teachers:
class_cour = pd.merge(classrooms, courses, on='CourseID', how='left')
tea_class_cour = pd.merge(teachers, class_cour, on='TeacherID', how='left')

### Group teachers, calculate the average degree in their corses and return in descending order:
result = tea_class_cour.groupby('TeacherID', as_index=False).agg({"FirstName": 'max', "LastName": 'max', 'degree':'mean'})
result = tea_class_cour.groupby(['TeacherID', 'FirstName', 'LastName'], as_index=False).agg({'degree':'mean'})
result.columns = ['TeacherID', 'Teacher_first_name', 'Teacher_last_name', 'Average_grade']
result.sort_values(ascending=False, by='Average_grade').dropna()

,TeacherID,Teacher_first_name,Teacher_last_name,Average_grade
21,22,Darick,Hess,81.432703
2,3,Jordyn,Baron,80.729494
4,5,Heather,Metcalf,80.440242
19,20,Zachary,Healy,79.365867
0,1,Kaylea,Monzingo,79.365537
17,18,Daniel,Bland,78.908281
23,24,Keegan,Price,78.814631
14,15,Conor,Mckittrick,78.699492
22,23,Maxwell,Romkee,78.673509
18,19,Terran,Beaty,78.495479


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [18]:
### Join tables of courses, departments, teachers and classrooms:
dep_cour = pd.merge(departments, courses, on='DepartmentID', how='left')
tea_dep_cour = pd.merge(teachers, dep_cour, on='TeacherID', how='left')
cla_tea_dep_cour = pd.merge(classrooms, tea_dep_cour, on='CourseID', how='right')

### Group courses together::
result = cla_tea_dep_cour.groupby(['CourseID', 'CourseName', 'DepartmentName','FirstName', 'LastName'], as_index=False).agg({'StudentID':'count'})
result.columns = ['CourseID', 'Course_name', 'Department_name', 'Teacher_first_name', 'Teacher_last_name', 'Number_of_students']
result

,CourseID,Course_name,Department_name,Teacher_first_name,Teacher_last_name,Number_of_students
0,1,English Begginers,English,Conor,Mckittrick,32
1,2,Advanced English,English,Conor,Mckittrick,24
2,3,Proffesional English,English,Conor,Mckittrick,38
3,4,Mathematics 101,Science,Amanda,Dworkin,18
4,5,Matematics Advanced,Science,Alexander,Watts,10
5,6,Algebra,Science,Alexander,Watts,12
6,7,Geometry,Science,Alexander,Watts,14
7,8,Chemistry,Science,Jeffrey,Batzel,24
8,9,Physics,Science,Alexander,Watts,12
9,10,Biology,Science,Jordyn,Baron,36


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [7]:
### First option per question:
### Join tables of students, classrooms and courses:
stu_cls = pd.merge(students, classrooms, on='StudentID', how='left')
stu_cls_cour = pd.merge(stu_cls, courses, on='CourseID', how='left')
stu_cls_cour.columns = ['StudentID', 'First_name', 'Last_name', 'Gender', 'CourseID', 'Grade', 'Course_name', 'DepartmentID', 'TeacherID']

### Gegerate a table to calculate the number of courses each student takes:
Num = stu_cls_cour.groupby('StudentID', as_index=False).agg({'CourseID':'count'})
Num.columns=['StudentID', 'Number_of_courses']

### Generate a table to claculate the average grade in all courses
GPA = stu_cls_cour.groupby('StudentID', as_index=False).agg({'Grade':'mean'})
GPA.columns=['StudentID', 'total_GPA']

### Merge all tables to show the number of courses, average per course as well as the total GPA:
a = pd.merge(stu_cls_cour, Num, on='StudentID', how='left')[['StudentID','First_name','Last_name', 'Number_of_courses', 'Course_name', 'Grade']]
answer = pd.merge(a, GPA, on="StudentID", how = 'left')
answer.head()


,StudentID,First_name,Last_name,Number_of_courses,Course_name,Grade,total_GPA
0,162,Elmira,Holston,3,Chemistry,65.987050,73.336714
1,162,Elmira,Holston,3,History of Arts,79.476271,73.336714
2,162,Elmira,Holston,3,Gymnastics,74.546820,73.336714
3,154,Sierra,Van Vorst,2,Algebra,84.137367,82.592693
4,154,Sierra,Van Vorst,2,R,81.048019,82.592693


In [8]:
### Second option per provided answer table:
### Join tables of students, classrooms, courses and departments:
stu_cls = pd.merge(students, classrooms, on='StudentID', how='left')
stu_cls_cour = pd.merge(stu_cls, courses, on='CourseID', how='left')
stu_cls_cour_dep = pd.merge(stu_cls_cour, departments, on='DepartmentID', how='left')
stu_cls_cour_dep.columns = ['StudentID', 'First_name', 'Last_name', 'Gender', 'CourseID', 'Grade', 'Course_name', 'DepartmentID', 'TeacherID', 'Department_name']

### Generate a table to calculate the number of courses each student takes:
Num = stu_cls_cour_dep.groupby(['StudentID', 'First_name', 'Last_name'], as_index=False).agg({'CourseID':'count'})
Num.columns=['StudentID', 'First_name', 'Last_name', 'Number_of_courses']

### Generate a table to claculate the average grade in all courses
GPA = stu_cls_cour_dep.groupby('StudentID', as_index=False).agg({'Grade':'mean'})
GPA.columns=['StudentID', 'total_GPA']

### Generate tables for each deaprtment to claculate the average grade of all courses in that department:
Arts_GPA=stu_cls_cour_dep.loc[(stu_cls_cour_dep['Department_name']=='Arts')].groupby(['StudentID'], as_index=False).agg({'Grade':'mean'})
Arts_GPA.columns=['StudentID', 'Arts']
English_GPA=stu_cls_cour_dep.loc[(stu_cls_cour_dep['Department_name']=='English')].groupby(['StudentID'], as_index=False).agg({'Grade':'mean'})
English_GPA.columns=['StudentID', 'English']
Science_GPA=stu_cls_cour_dep.loc[(stu_cls_cour_dep['Department_name']=='Science')].groupby(['StudentID'], as_index=False).agg({'Grade':'mean'})
Science_GPA.columns=['StudentID', 'Science']
Sports_GPA=stu_cls_cour_dep.loc[(stu_cls_cour_dep['Department_name']=='Sport')].groupby(['StudentID'], as_index=False).agg({'Grade':'mean'})
Sports_GPA.columns=['StudentID', 'Sports']

### Join all tables to present the information about each student:
a = pd.merge(Num, English_GPA, on='StudentID', how='left')
b = pd.merge(a, Arts_GPA, on='StudentID', how='left')
c = pd.merge(b, Science_GPA, on='StudentID', how='left')
d = pd.merge(c, Sports_GPA, on='StudentID', how='left')
result = pd.merge(d, GPA, on='StudentID', how='left')
result.head()


,StudentID,First_name,Last_name,Number_of_courses,English,Arts,Science,Sports,total_GPA
0,1,Alexandria,Heller,4,NaN,86.677535,79.557508,NaN,81.337515
1,2,Cassie,Liska,5,NaN,79.542738,78.867115,NaN,79.137364
2,3,Hayley,Frank,3,83.972245,NaN,76.204427,NaN,78.793700
3,4,Maxwell,Kotch,3,70.929801,69.356144,74.157066,NaN,71.481003
4,5,Dylan,Boehm,3,NaN,64.494845,100.000000,65.661106,76.718650
